Peg-solitaire Game
---------
Imen AYADI
-----------

In [72]:
import numpy as np
import itertools
import cmath
import cvxopt
import cvxopt.glpk
from cvxopt import matrix
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib notebook

Notation:
---
* n = number of squares of the game's board (we take n = 33)
* m = number of possible jumps (if n = 33 then m = 76)
* If a square is occupied by a peg , we associate to it the value 1 , otherwise 0

Method 0 : with no IP formulation nor backtracking): 
---------------------------------------------------------------------------

My first idea to modelize the game (I didn't find it in articles or books) is to consider an oriented graph G = (E,V) whose nodes are the configurations of the game and such that a configuration p is linked to a configuration p' iif there is a possible jump that transforms the configuration p to the configuration p'

Note that the term 'possible configuration' does not mean that is reachable: some nodes have no neighbors

Every jump (vertice) has a weight equal to 1

Our goal is to reach the final configuration pf with the fewest moves (giving an initial configuration ps). Then, we want to find the shortest path from ps to pf.

Let us estimate the computational complexity of this algorithm:
* if we note n the number of the squares of the board (usually n=33), the number of the possible
  configurations could be |E| = 2^n - 2
* m is the number of possible jumps : So, |V|=m          (m < 4*n ) ; in case of n=33 , m = 76
* We construct a dictionary of neighbors containing all the possible configurations(nodes) and all jumps(vertices): we generate
  all possible binary n-dimension vectors (so in total 2^n) , then , for each two vectors, we test if there is a possible jump    transforming the vector with the higher number of 1's (i.e of pegs) to the other.
* Because the nomber of vertices (~ constant * n ) is lower than the number of edges (~ 2^n) , then the Floyd-Warshall algorithm is more
  efficient then than the Dijsktra algorithm to find to the shortest path:In fact the complexity of Floyd-Warshall is O(|V|^3) 
  while the complexity of Dijsktra is O((|V|+|E|) log(|V|)).
  Therefore, the computational complexity of the algorithm of searching the shortest path is O(m^3) = O(n^3)
* The essential drawback of this method is the complexity of the modelization (construction of the graph) not the solving's complexity

>I tried to implement this method but when executing the code that just enumerates the 2^33 - 2 possible configurations, the notebook blocked. Therefore, this idea is useless.

Method 1
-----

Formulate it as an IP problem. 

This is the implementation of IP1.

IP1 is the IP formulation of Peg Solitaire, we can try to solve it directly. However
this problem is unsolvable for a bigger checkerboard or more steps, thus we try 
to solve it with branch and bound.

$$
\begin{split}
    IP1: & \text{find} \quad & (x^1, ... , x^l) &\\
               & s.t.       &A(x^1+ ... + x^l) = p_s - p_f          ,&                    (1)\\
               &            &0 \le p_s - (x^1, ... , x^k) \le 1 , \quad &\forall k \in \{1, ..., l\}  ,       &(2)\\
               &            &x^k_1 + ... + x^k_m = 1 , \quad           &\forall k \in \{1, ..., l\}  ,        &(3)\\
               &            &x^k \in \{0,1\}^m   & \forall k ,               &(4)\\
\end{split}
$$

Solving the entire problem will spending too much time, but it's acceptable to solve until 20th step.

For the convenience of repeating our work, we recommend to set the ending step less thant 18.

In [73]:
#Defining 'm' parameter
m = 76 # It corresponds to the number of possible jumps in an English peg solitaire problem

In [74]:
def starting_and_finishing_configurations(initial_hole_position):
    #Initial representation
    ps = np.ones(33)
    ps[initial_hole_position]=0 #this hole has no peg in the starting representation

    #Final representation
    pf = np.zeros(33)
    pf[initial_hole_position]=1 #Central hole is the only filled hole in the final representation
    
    return (ps,pf)

In [75]:
#Defining 'l' parameter
def nbr_moves(initial_hole_position):
    ps,pf=starting_and_finishing_configurations(initial_hole_position)
    return int(np.sum(ps)-np.sum(pf))

#Defining cost_matrix
def cost_matrix(initial_hole_position):
    l=nbr_moves(initial_hole_position)
    return  matrix(np.zeros(l*m), tc='d')

#Defining binary variables
def Bin_var(initial_hole_position):
    l = nbr_moves(initial_hole_position)
    return set(range(l*m))

In [76]:
#Implementing matrix A 

#First, implementing A1
A10= np.array([[1,0,0,0,0], [1,1,0,0,0], [-1,1,1,0,0], [0,-1,1,1,0], [0,0,-1,1,1], [0,0,0,-1,1], [0,0,0,0,-1]])

A11 = np.zeros((33,1))
A11[0]=1
A11[1]=1
A11[2]=-1

A12 = np.zeros((33,1))
A12[3]=1
A12[4]=1
A12[5]=-1

A13 = np.vstack((np.zeros((6,5)), A10, np.zeros((20,5))))

A14 = np.vstack((np.zeros((13,5)), A10, np.zeros((13,5))))

A15 = np.vstack((np.zeros((20,5)), A10, np.zeros((6,5))))
A16 = np.zeros((33,1))
A16[27]=1
A16[28]=1
A16[29]=-1

A17 = np.zeros((33,1))
A17[30]=1
A17[31]=1
A17[32]=-1

A1 = np.hstack((A11,A12,A13,A14,A15,A16,A17))


#Then, implementing A2
A20= np.array([[-1,0,0,0,0], [1,-1,0,0,0], [1,1,-1,0,0], [0,1,1,-1,0], [0,0,1,1,-1], [0,0,0,1,1], [0,0,0,0,1]])

A21 = np.zeros((33,1))
A21[0]=-1
A21[1]=1
A21[2]=1

A22 = np.zeros((33,1))
A22[3]=-1
A22[4]=1
A22[5]=1

A23 = np.vstack((np.zeros((6,5)), A20, np.zeros((20,5))))

A24 = np.vstack((np.zeros((13,5)), A20, np.zeros((13,5))))

A25 = np.vstack((np.zeros((20,5)), A20, np.zeros((6,5))))

A26 = np.zeros((33,1))
A26[27]=-1
A26[28]=1
A26[29]=1

A27 = np.zeros((33,1))
A27[30]=-1
A27[31]=1
A27[32]=1

A2 = np.hstack((A21,A22,A23,A24,A25,A26,A27))

#Then, implementing A3
A31 = np.vstack((-np.eye(3), np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((22,3))))

A32 = np.vstack((np.zeros((3,3)), -np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((4,3)), np.eye(3), np.zeros((15,3))))

A33 = np.vstack((np.zeros((6,7)), -np.eye(7), np.eye(7), np.eye(7), np.zeros((6,7))))

A34 = np.vstack((np.zeros((15,3)), -np.eye(3), np.zeros((4,3)), np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((3,3))))

A35 = np.vstack((np.zeros((22,3)), -np.eye(3), np.zeros((2,3)), np.eye(3), np.eye(3)))

A3 = np.hstack((A31,A32,A33,A34,A35))

#Then, implementing A4
A41 = np.vstack((np.eye(3), np.eye(3), np.zeros((2,3)), -np.eye(3), np.zeros((22,3))))

A42 = np.vstack((np.zeros((3,3)), np.eye(3), np.zeros((2,3)), np.eye(3), np.zeros((4,3)), -np.eye(3), np.zeros((15,3))))

A43 = np.vstack((np.zeros((6,7)), np.eye(7), np.eye(7), -np.eye(7), np.zeros((6,7))))

A44 = np.vstack((np.zeros((15,3)), np.eye(3), np.zeros((4,3)), np.eye(3), np.zeros((2,3)), -np.eye(3), np.zeros((3,3))))

A45 = np.vstack((np.zeros((22,3)), np.eye(3), np.zeros((2,3)), np.eye(3), -np.eye(3)))

A4 = np.hstack((A41,A42,A43,A44,A45))

# Concatenating A_i horizontally to constitute A
A = np.hstack((A1,A2,A3,A4)).astype(int)



In [77]:
#Equality constraints 

def equality_constraints(initial_hole_position):
    l= nbr_moves(initial_hole_position)
    ps,pf=starting_and_finishing_configurations(initial_hole_position)
    # Let us consider first equality constraints ( refer to constraint (1) )
    E11 = np.hstack((np.eye(m) for i in range(l)))
    E1 = np.matmul(A,E11)

    e1 = (ps - pf).reshape((33,1))

    # Let us consider the second king of equality constraints ( refer to constraint (3) )
    E2 = np.zeros((l,l*m))
    for i in range(l):
        E2[i][(i*m):((i+1)*m)] = np.ones((1,m))

    e2 = np.ones((l,1))

    # Concatenating vertically equality matrices
    E = np.vstack((E1,E2))
    E = matrix(E, tc='d')

    e = np.vstack((e1,e2))
    e = matrix(e, tc='d')
    return E,e

In [78]:
# Inequality constraints ( refer to constraint (2) )
# let us define inequality matrices

def inequality_constraints(initial_hole_position):
    l=nbr_moves(initial_hole_position)
    ps,pf=starting_and_finishing_configurations(initial_hole_position)

    # Let us consider first the " >= 0 " side
    for k in range(l):
        I1k1 = np.hstack((np.eye(m) for i in range(k+1)))
        if k == l-1:
            I1k = I1k1
        else:
            I1k2 = np.hstack((np.zeros((m,m)) for j in range(k+1,l)))
            I1k = np.hstack((I1k1,I1k2))
        A1k = np.matmul(A,I1k)
        if k == 0:
            I1 = A1k
        else:
            I1 = np.vstack((I1, A1k))

    i10 = ps.reshape((33,1))
    i1 = np.vstack((i10 for  i in range(l)))

    # Let us consider first the " <= 0 " side
    for k in range(l):
        I2k1 = np.hstack((-1*np.eye(m) for i in range(k+1)))
        if k == l-1:
            I2k = I2k1
        else:
            I2k2 = np.hstack((np.zeros((m,m)) for j in range(k+1,l)))
            I2k = np.hstack((I2k1,I2k2))
        A2k = np.matmul(A,I2k)
        if k == 0:
            I2 = A2k
        else:
            I2 = np.vstack((I2, A2k))

    i20 = np.ones((33,1)) - ps.reshape((33,1))
    i2 = np.vstack((i20 for  i in range(l)))

    #Concatenating inequality matrices
    I = np.vstack((I1,I2))
    I = matrix(I, tc='d')

    i = np.vstack((i1,i2))
    i = matrix(i, tc='d')
    
    return I,i

!CAUTION!
----
To obtain the solution via method 1 , you can execute the following lines of code BUT IT TAKES VERY VERY LONG TIME
Then, DO NOT EXECUTE IT !!!

In [ ]:
# Solving the problem

initial_hole_position=16 # in the center 
C=cost_matrix(initial_hole_position)
B=Bin_var(initial_hole_position)
E,e=equality_constraints(initial_hole_position)
I,i=inequality_constraints(initial_hole_position)

(status, optimal_x) = cvxopt.glpk.ilp(c=C,G=I,h=i,A=E,b=e,B=B) 


* we have l x m = 31 x 76 = 2356 variables. Furthermore, we have n + l = 33 + 31 = 64 equality constraints and 2 x l x n = 2 x 31 x 33 = 2046 inequality constraints.
* Since there are 2356 variables in our model, we can say that there exists 2^2356 feasible peg configuration !
* Therefore, the size of the integer programming problem is huge and so it is hardto solve the problem by personal computers. 
* Using cvxopt with my personal computer, the algorithm kept searching for the solution during 5 hours before I interrupted it.

> Conclusion:

The Method 1 is not helpful. We should look for another approach to deal with the problem : an alternative way that eliminates the unuseful recursive excecutions by pruning as much as possible the blocking branchs in order to improves the backtracking search

--------------------

Method 2
------

To help with the speed, we use Branch and bound + hash_table + back_tracking

This method is based on the "Article 2" [Integer Programming Based Algorithms for Peg Solitaire Problems](https://hugues-talbot.github.io/files/Peg_Solitaire_2.pdf).

The pipeline can be described as following:

1. For each jump $j$, calculate its upper_bound by solving the IP problem RUBj.

    RUBj is the relaxed problem of IP1. It's easy to prove that a necessary condition for an IP1 to be feasible
    is that RUBj is feasible. 
    
    And the number of each jump in RUBj is greater or equal to the number of each jump in IP1 respectfully.
   
   $$
   \begin{split}
       RUBj: & \max. \quad &x_j \\
                  & s.t.                &Ax = p_s - p_f    \\
                  && x = (x_1, ..., x_m) \text{ non negative integers}
   \end{split}
   $$
   
2. Use branch and bound to solve the original problem

In [79]:
# Here we shuffle columns of A, in order to explore randomly moves among the 76 possible ones
# When fixing seed=82, we have better performances

np.random.seed(seed=82)
s = np.arange(A.shape[1])
np.random.shuffle(s)
A = A[:,s]


# Defining matrices for RUB integer program
Erub = matrix(A, tc='d')
    
Irub = -1*np.eye(m)
Irub = matrix(Irub, tc='d')

irub = np.zeros((m,1))
irub = matrix(irub, tc='d')

Srub = set(range(m))


# Taking an example of 3 pagoda functions that were given in the original paper
pagval1_array = np.array([1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,1])
pagval2_array = np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0])
pagval3_array = np.array([0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0])

# Computing pagoda value for the finishing configuration
def pagval_end(initial_hole_position):
    _,pf=starting_and_finishing_configurations(initial_hole_position)
    pagval1_end = np.matmul(pagval1_array.T,pf)
    pagval2_end = np.matmul(pagval2_array.T,pf)
    pagval3_end = np.matmul(pagval3_array.T,pf)
    return pagval1_end,pagval2_end,pagval3_end
    

In [80]:
# Defining a function that solves RUB program given the jump j, the starting and finishing configurations

def solveRUB(jump, ps, pf):
    cost = np.zeros(m)
    cost[jump] = -1
    cost = matrix(cost, tc='d')
    
    erub = matrix((ps - pf).reshape((33,1)), tc='d')
    
    (status, optimal_x) = cvxopt.glpk.ilp(c=cost,G=Irub,h=irub,A=Erub,b=erub,I=Srub)
    
    if optimal_x:
        x_jump = np.matmul(cost.T, optimal_x)
        x_jump = x_jump[0][0]
        return(-1*x_jump)
    else:
        return None

In [81]:
# Defining a function which states if a  configuration p is legal or not meaning weither it is a binary vector or not
def legal(p):
    for i in range(len(p)):
        if p[i]<0 or p[i]>1:
            return False
    return True

In [82]:
# Defining a recursive function that takes in input the intermediate configuration "start", the finishing configuration "end
# which remains always the same, the list of upper bounds, the dictionary of searches already computed, and the path which is
# updated at each stage.

def search(start, end, upper_bound, path, dictionary,initial_hole_position):
    pagval1_end,pagval2_end,pagval3_end=pagval_end(initial_hole_position)
    
    reste = int(np.sum(start)-np.sum(end))
    
    if reste <= 0:
        return(np.array_equal(start,end))
    
    else:
        pagval1_t = np.matmul(pagval1_array.T,start)
        pagval2_t = np.matmul(pagval2_array.T,start)
        pagval3_t = np.matmul(pagval3_array.T,start)

        if pagval1_t < pagval1_end or pagval2_t < pagval2_end or pagval3_t < pagval3_end:
            return False
        
        else:
            
            for j in range(m):
                
                optimal_x = solveRUB(j, start, end)
                
                if optimal_x is None:
                    return False
                
                else:
                    if upper_bound[j] <= 0:
                        continue

                    u_j = np.zeros(m)
                    u_j[j] = 1

                    move = start - np.matmul(A,u_j)

                    if legal(move):
                        start = move

                        upper_bound[j] = upper_bound[j] - 1
                        path.append(s[j])

                        mytuple = hash(tuple(start))

                        if mytuple in dictionary.keys():
                            y = dictionary[mytuple]
                        else:
                            y = search(start, end, upper_bound, path, dictionary,initial_hole_position)
                            dictionary[mytuple] = y

                        if y == 1:
                            return True
                        else:
                            upper_bound[j] = upper_bound[j] + 1
                            start = start + np.matmul(A,u_j)
                            path.pop()

            return False

In [83]:
# Defining the backtrack peg solver

def backtrack_peg_solver(initial_hole_position):
    dictionary = {}
    upper_bound = np.zeros(m)
    path = []
    ps,pf=starting_and_finishing_configurations(initial_hole_position)
    
    for jump in range(m):
        optimal_x = solveRUB(jump, ps, pf)
        if optimal_x is None:
            print("Infeasible problem")
            return None
        else:
            upper_bound[jump] = optimal_x
    
    if search(ps, pf, upper_bound, path, dictionary,initial_hole_position) == 0:
        print("Infeasible problem")
        return None
    
    print("Feasible problem")
    return path

In [84]:
initial_hole_position=16 # in the center
start = time.time()
solution = backtrack_peg_solver(initial_hole_position)
end = time.time()
print("The list of moves composing the feasible solution is : \n-->", solution)
print("The elapsed computing time is :", round(end - start,2), "s")

Feasible problem
The list of moves composing the feasible solution is : 
--> [61, 47, 13, 54, 32, 55, 12, 32, 63, 12, 21, 34, 32, 56, 72, 62, 45, 2, 22, 57, 41, 19, 57, 25, 50, 3, 5, 25, 67, 53, 29]
The elapsed computing time is : 5.63 s


>The method 2 is an improved version of method 1. Therefore, it should provide as a solution that satistifies the IP1.

In [85]:
#Graphical representation of the solution 

#As A has been shuffled before, we recreate the initial A matrix
A = np.hstack((A1,A2,A3,A4)).astype(int)

def print_config(p):
    pp=[]
    for i in range(33):
        if int(p[i])==0:
            pp.append('o')
        else:
            pp.append('\u25CF') 
    ch='  '
    line=''
    for i in range(2):
        l=''
        for j in range(3):
            l=l+pp[j+i*3]
        line=ch+l
        print(line)
    for i in range(3):
        r=''
        for j in range(7):
            r=r+pp[i*7+j+6]
        print(r)
    line=''
    for i in range(2):
        l=''
        for j in range(3):
            l=l+pp[j+i*3+27]
        line=ch+l
        print(line)

print_config(ps)

def print_solution(solution,intial_hole_position):
    ps,pf=starting_and_finishing_configurations(initial_hole_position)
    #This function prints the solution's sequence
    print('------Initial Configuration' + '------\n')
    print_config(ps)
    p = ps
    zeros = np.zeros(m)
    for i in range(len(solution)-1):
        zeros[solution[i]] = 1
        p = p - np.matmul(A,zeros)
        print('-------------Move ' + str(i+1) +'-------------\n')
        print_config(p)
        zeros = np.zeros(m)
    print('------Final Configuration' + '------\n')
    print_config(pf)

print_solution(solution,16)

  ●●●
  ●●●
●●●●●●●
●●●o●●●
●●●●●●●
  ●●●
  ●●●
------Initial Configuration------

  ●●●
  ●●●
●●●●●●●
●●●o●●●
●●●●●●●
  ●●●
  ●●●
-------------Move 1-------------

  ●●●
  ●o●
●●●o●●●
●●●●●●●
●●●●●●●
  ●●●
  ●●●
-------------Move 2-------------

  ●●●
  ●o●
●●●●●●●
●●●o●●●
●●●o●●●
  ●●●
  ●●●
-------------Move 3-------------

  ●●●
  ●o●
●●●●●●●
●●●o●●●
●oo●●●●
  ●●●
  ●●●
-------------Move 4-------------

  ●●●
  ●o●
●●●●●●●
●●●o●●●
●o●●●●●
  o●●
  o●●
-------------Move 5-------------

  ●●●
  ●o●
●●●●●●●
●●●o●●●
●●oo●●●
  o●●
  o●●
-------------Move 6-------------

  ●●●
  ●o●
●●●●●●●
●●●o●●●
●●o●●●●
  oo●
  oo●
-------------Move 7-------------

  ●●●
  ●o●
●●●●●●●
●●●o●●●
oo●●●●●
  oo●
  oo●
-------------Move 8-------------

  ●●●
  ●o●
●●●●●●●
●●●o●●●
o●oo●●●
  oo●
  oo●
-------------Move 9-------------

  ●●●
  ●o●
o●●●●●●
o●●o●●●
●●oo●●●
  oo●
  oo●
-------------Move 10-------------

  ●●●
  ●o●
o●●●●●●
o●●o●●●
oo●o●●●
  oo●
  oo●
-------------Move 11-------------

  ●●●
  ●o●
●

Improving the Method 2 : Using symmetries
------

In order to exploit symmetry, there must be a means representing congruent boards by a unique identifier. 

There are 4 simple rotations whos center is the center of the board and the angle is 90° , 180° , 270° and 0°(identity).
There are 2 axial symmetries whose axes are either the horizontal line passing by the center and the vertical line passing by the center. 
So, by composing rotations and axial symmetries , we obtain in total 8 isometries that transform a configuration to an equivalent one . 

In fact,the board can be represented by the dihedral group D

Then, the number of explored configurations could be divided by 8 >> Improve the backtracking search

In [90]:
# We introduce a Cartesian coordinate system whose center is the center of the board (i.e. the square n° 16 ) 

coordonates={0:(-1,3),1:(0,3),2:(1,3),3:(-1,2),4:(0,2),5:(1,2),6:(-3,1),7:(-2,1),8:(-1,1),9:(0,1),10:(1,1),11:(2,1),
             12:(3,1),13:(-3,0),14:(-2,0),15:(-1,0),16:(0,0),17:(1,0),18:(2,0),19:(3,0),20:(-3,-1),21:(-2,-1),22:(-1,-1),
             23:(0,-1),24:(1,-1),25:(2,-1),26:(3,-1),27:(-1,-2),28:(0,-2),29:(1,-2),30:(-1,-3),31:(0,-3),32:(1,-3)}

reverse_coordonates={(-1,3):0,(0,3):1,(1,3):2,(-1,2):3,(0,2):4,(1,2):5,(-3,1):6,(-2,1):7,(-1,1):8,(0,1):9,(1,1):10,(2,1):11,
             (3,1):12,(-3,0):13,(-2,0):14,(-1,0):15,(0,0):16,(1,0):17,(2,0):18,(3,0):19,(-3,-1):20,(-2,-1):21,(-1,-1):22,
             (0,-1):23,(1,-1):24,(2,-1):25,(3,-1):26,(-1,-2):27,(0,-2):28,(1,-2):29,(-1,-3):30,(0,-3):31,(1,-3):32}

In [96]:
#generate the 8 equivalent configurations to a given configuration p

def sym1(p):
    q=[]
    for j in range(33):
        x,y=coordonates[j]
        q.append(p[reverse_coordonates[(-x,-y)]])
    return q

def sym2(p):
    q=[]
    for j in range(33):
        x,y=coordonates[j]
        q.append(p[reverse_coordonates[(-x,y)]])
    return q

def sym3(p):
    q=[]
    for j in range(33):
        x,y=coordonates[j]
        q.append(p[reverse_coordonates[(x,-y)]])
    return q

def sym4(p):
    q=[]
    for j in range(33):
        x,y=coordonates[j]
        q.append(p[reverse_coordonates[(y,x)]])
    return q

def sym5(p):
    q=[]
    for j in range(33):
        x,y=coordonates[j]
        q.append(p[reverse_coordonates[(y,-x)]])
    return q

def sym6(p):
    q=[]
    for j in range(33):
        x,y=coordonates[j]
        q.append(p[reverse_coordonates[(-y,-x)]])
    return q

def sym7(p):
    q=[]
    for j in range(33):
        x,y=coordonates[j]
        q.append(p[reverse_coordonates[(-y,x)]])
    return q

# sym = identity

In [97]:
#Modify the function th function "search" with taking into account the symmetries of the board
def search_with_symmetries(start, end, upper_bound, path, dictionary,initial_hole_position):
    pagval1_end,pagval2_end,pagval3_end=pagval_end(initial_hole_position)
    
    reste = int(np.sum(start)-np.sum(end))
    
    if reste <= 0:
        return(np.array_equal(start,end))
    
    else:
        pagval1_t = np.matmul(pagval1_array.T,start)
        pagval2_t = np.matmul(pagval2_array.T,start)
        pagval3_t = np.matmul(pagval3_array.T,start)

        if pagval1_t < pagval1_end or pagval2_t < pagval2_end or pagval3_t < pagval3_end:
            return False
        
        else:
            
            for j in range(m):
                
                optimal_x = solveRUB(j, start, end)
                
                if optimal_x is None:
                    return False
                
                else:
                    if upper_bound[j] <= 0:
                        continue

                    u_j = np.zeros(m)
                    u_j[j] = 1

                    move = start - np.matmul(A,u_j)

                    if legal(move):
                        start = move

                        upper_bound[j] = upper_bound[j] - 1
                        path.append(s[j])

                        mytuple = hash(tuple(start))
                        
                        if len (path)>=16:
                            mytuple1 = hash(tuple(sym1(start)))
                            mytuple2 = hash(tuple(sym2(start)))
                            mytuple3 = hash(tuple(sym3(start)))
                            mytuple4 = hash(tuple(sym4(start)))
                            mytuple5 = hash(tuple(sym5(start)))
                            mytuple6 = hash(tuple(sym6(start)))
                            mytuple7 = hash(tuple(sym7(start)))
                            
                            #We choose the number that is the smallest to represent this board,
                            #and all congruent boards will also result in the same unique identifie    
                            mytuple = min([mytuple,mytuple1,mytuple2,mytuple3,mytuple4,mytuple5,mytuple6,mytuple7])
                            
                        if mytuple in dictionary.keys():
                            y = dictionary[mytuple]
                        else:
                            y = search(start, end, upper_bound, path, dictionary, initial_hole_position)
                            dictionary[mytuple] = y
                            
                        if y == 1:
                            return True
                        else:
                            upper_bound[j] = upper_bound[j] + 1
                            start = start + np.matmul(A,u_j)
                            path.pop()

            return False
        
# Defining the backtrack peg solver

def backtrack_peg_solver_with_symmetries(initial_hole_position):
    dictionary = {}
    upper_bound = np.zeros(m)
    path = []
    ps,pf=starting_and_finishing_configurations(initial_hole_position)
    
    for jump in range(m):
        optimal_x = solveRUB(jump, ps, pf)
        if optimal_x is None:
            print("Infeasible problem")
            return False
        else:
            upper_bound[jump] = optimal_x
    
    if search_with_symmetries(ps, pf, upper_bound, path, dictionary,initial_hole_position) == 0:
        print("Infeasible problem")
        return False
    
    print("Feasible problem")
    return path

In [ ]:
initial_hole_position=16 # in the center
start = time.time()
solution = backtrack_peg_solver_with_symmetries(initial_hole_position)
end = time.time()
print("The list of moves composing the feasible solution is : \n-->", solution)
print("The elapsed computing time is :", round(end - start,2), "s")